# CEND Pipeline - Interactive Visualization

Este notebook simula o pipeline completo de reconstrução de neurônios 3D do CEND, permitindo visualizar cada etapa e ajustar parâmetros interativamente.

## Pipeline Overview

1. **Load Data**: Carregar volume 3D
2. **Preprocessing**: Gaussian + Minimum filtering
3. **Multi-scale Filtering**: Tubular filtering (Yang/Frangi/Kumar)
4. **Segmentation**: Thresholding adaptativo + denoising
5. **Distance Fields**: Pressure e Thrust fields
6. **Terminal Detection**: Local maxima no thrust field
7. **Skeletonization**: Dijkstra-based path finding
8. **Graph Construction**: MST + pruning
9. **Export**: SWC format

##  Setup & Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import skeletonize
import logging
import gc

# CEND imports (nova estrutura)
from cend.io import load_3d_volume
from cend.processing.multiscale import multiscale_filtering
from cend.core.segmentation import (
    adaptive_mean_mask,
    grey_morphological_denoising,
    morphological_denoising
)
from cend.core.distance_fields import DistanceFields
from cend.core.skeletonization import generate_skeleton_from_seed
from cend.core.vector_fields import create_maxima_image
from cend.structures import Graph

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Configurar matplotlib
plt.rcParams['figure.figsize'] = (15, 5)
plt.rcParams['figure.dpi'] = 100

print("✓ Imports carregados com sucesso!")

### Helper Functions para Visualização

In [ ]:
def plot_3d_projections(volume, title="Volume Projections", cmap='gray', vmin=None, vmax=None):
    """Plota projeções 3D (max, mean) nas três dimensões."""
    fig, axes = plt.subplots(2, 3, figsize=(15, 15))
    
    projections = [
        ('Max', lambda v, ax: np.max(v, axis=ax)),
        ('Mean', lambda v, ax: np.mean(v, axis=ax)),
    ]
    
    axes_names = ['Z (depth)', 'Y (height)', 'X (width)']
    
    for row, (proj_name, proj_func) in enumerate(projections):
        for col, axis in enumerate([0, 1, 2]):
            ax = axes[row, col]
            projection = proj_func(volume, axis)
            im = ax.imshow(projection, cmap=cmap, vmin=vmin, vmax=vmax)
            ax.set_title(f'{proj_name} projection - {axes_names[col]}')
            ax.axis('off')
            plt.colorbar(im, ax=ax, fraction=0.046)
    
    fig.suptitle(title, fontsize=16, y=0.995)
    plt.tight_layout()
    plt.show()


def plot_slice_comparison(vol1, vol2, slice_idx=None, titles=('Before', 'After'), 
                          axis=0, cmap='gray', figsize=(12, 5)):
    """Compara uma fatia de dois volumes lado a lado."""
    if slice_idx is None:
        slice_idx = vol1.shape[axis] // 2
    
    fig, axes = plt.subplots(1, 2, figsize=figsize)
    
    # Selecionar fatia baseado no eixo
    if axis == 0:
        slice1, slice2 = vol1[slice_idx], vol2[slice_idx]
    elif axis == 1:
        slice1, slice2 = vol1[:, slice_idx], vol2[:, slice_idx]
    else:
        slice1, slice2 = vol1[:, :, slice_idx], vol2[:, :, slice_idx]
    
    im1 = axes[0].imshow(slice1, cmap=cmap)
    axes[0].set_title(f'{titles[0]} (slice {slice_idx})')
    axes[0].axis('off')
    plt.colorbar(im1, ax=axes[0])
    
    im2 = axes[1].imshow(slice2, cmap=cmap)
    axes[1].set_title(f'{titles[1]} (slice {slice_idx})')
    axes[1].axis('off')
    plt.colorbar(im2, ax=axes[1])
    
    plt.tight_layout()
    plt.show()


def plot_histogram(volume, title="Intensity Histogram", bins=50):
    """Plota histograma de intensidades."""
    plt.figure(figsize=(10, 4))
    plt.hist(volume.ravel(), bins=bins, alpha=0.7, edgecolor='black')
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')
    plt.title(title)
    plt.grid(alpha=0.3)
    plt.show()


def print_stats(volume, name="Volume"):
    """Imprime estatísticas do volume."""
    print(f"\n{name} Statistics:")
    print(f"  Shape: {volume.shape}")
    print(f"  Dtype: {volume.dtype}")
    print(f"  Min: {volume.min():.4f}")
    print(f"  Max: {volume.max():.4f}")
    print(f"  Mean: {volume.mean():.4f}")
    print(f"  Std: {volume.std():.4f}")
    print(f"  Non-zero voxels: {np.count_nonzero(volume)} ({100*np.count_nonzero(volume)/volume.size:.2f}%)")

print("✓ Helper functions definidas!")

---

## Configuração de Parâmetros

Ajuste os parâmetros do pipeline aqui:

In [ ]:
# ========================
# PARÂMETROS DO PIPELINE
# ========================

# Dados de entrada
DATA_DIR = "../data/OP_1"  # Caminho para os TIFFs
DATASET_NUMBER = 1

# Coordenada do ponto raiz/seed (z, y, x)
ROOT_COORD = (0, 429, 31)  # Ajuste conforme seu dataset

# Parâmetros de filtragem
FILTER_TYPE = "yang"  # "yang", "frangi", "kumar", ou "sato"
SIGMA_MIN = 1.0
SIGMA_MAX = 2.0
SIGMA_STEP = 0.5
NEURON_THRESHOLD = 0.05

# Parâmetros de skeletonização
MAXIMAS_MIN_DIST = 2  # Ordem para detecção de maxima local

# Parâmetros de graph/MST
PRUNING_THRESHOLD = 10  # Comprimento mínimo de branch (0 = desabilita)
SMOOTHING_FACTOR = 0.8
NUM_POINTS_PER_BRANCH = 15

# Visualização
SLICE_TO_SHOW = None  # None = meio do volume
SHOW_PROJECTIONS = True  # Mostrar projeções 3D completas

print("✓ Parâmetros configurados!")
print(f"  Dataset: {DATA_DIR}")
print(f"  Filter: {FILTER_TYPE}, sigma=[{SIGMA_MIN}, {SIGMA_MAX}], step={SIGMA_STEP}")
print(f"  Root coord: {ROOT_COORD}")

---

## Step 1: Load 3D Volume

Carrega o stack de imagens TIFF como um volume 3D numpy array.

In [ ]:
# Carregar volume 3D
print(f"Loading volume from {DATA_DIR}...")
volume_original = load_3d_volume(DATA_DIR)
print(f"✓ Volume loaded: {volume_original.shape}")

# Mostrar estatísticas
print_stats(volume_original, "Original Volume")

# Visualizar
if SHOW_PROJECTIONS:
    plot_3d_projections(volume_original, title="Original Volume - Max/Mean/Min Projections")
else:
    # Mostrar apenas uma fatia
    slice_idx = SLICE_TO_SHOW if SLICE_TO_SHOW else volume_original.shape[0] // 2
    plt.figure(figsize=(10, 8))
    plt.imshow(volume_original[slice_idx], cmap='gray')
    plt.title(f'Original Volume - Slice {slice_idx}')
    plt.colorbar()
    plt.axis('off')
    plt.show()

# Histograma de intensidades
plot_histogram(volume_original, "Original Volume - Intensity Distribution")

---

## Step 2: Preprocessing

Aplica Gaussian filter seguido de Minimum filter para remover ruído e melhorar a qualidade.

In [ ]:
# Copiar para não modificar o original
volume = volume_original.copy()

# Aplicar Gaussian filter
print("Applying Gaussian filter (sigma=1.0)...")
gauss_filtered = ndi.gaussian_filter(volume, 1.0)

# Aplicar Minimum filter
print("Applying Minimum filter (size=2)...")
min_filtered = ndi.minimum_filter(gauss_filtered, 2)

# Zerar voxels com valor 0 no minimum filter
volume[min_filtered == 0] = 0

# del gauss_filtered, min_filtered
# gc.collect()

print("✓ Preprocessing done!")
print_stats(volume, "Preprocessed Volume")

# Comparar antes e depois
plot_slice_comparison(
    volume_original, volume,
    titles=('Original', 'After Preprocessing'),
    cmap='gray'
)

# Histogramas lado a lado
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(volume_original.ravel(), bins=50, alpha=0.7, edgecolor='black')
axes[0].set_title('Original Histogram')
axes[0].set_xlabel('Intensity')
axes[0].set_ylabel('Frequency')
axes[0].grid(alpha=0.3)

axes[1].hist(volume.ravel(), bins=50, alpha=0.7, edgecolor='black', color='orange')
axes[1].set_title('Preprocessed Histogram')
axes[1].set_xlabel('Intensity')
axes[1].set_ylabel('Frequency')
axes[1].grid(alpha=0.3)
plt.tight_layout()
plt.show()

---

##  Step 3: Multi-scale Tubular Filtering

Aplica filtros Hessian-based para realçar estruturas tubulares (neuritos) em múltiplas escalas.

In [ ]:
# Aplicar multi-scale filtering
print(f"Applying {FILTER_TYPE} tubular filter...")
print(f"  Sigma range: [{SIGMA_MIN}, {SIGMA_MAX}], step={SIGMA_STEP}")
print(f"  Threshold: {NEURON_THRESHOLD}")

img_filtered = multiscale_filtering(
    volume=volume,
    sigma_range=(SIGMA_MIN, SIGMA_MAX, SIGMA_STEP),
    filter_type=FILTER_TYPE,
    neuron_threshold=NEURON_THRESHOLD,
    dataset_number=DATASET_NUMBER
)

# gc.collect()
print("✓ Multi-scale filtering done!")
print_stats(img_filtered, "Filtered Volume")

# Visualizar resultado
if SHOW_PROJECTIONS:
    plot_3d_projections(img_filtered, 
                       title=f"{FILTER_TYPE.capitalize()} Filter Response - Max/Mean/Min Projections",
                       cmap='hot')
    
# Comparação antes/depois
plot_slice_comparison(
    volume, img_filtered,
    titles=('Preprocessed', f'{FILTER_TYPE.capitalize()} Filtered'),
    cmap='hot'
)

# Histograma da resposta do filtro
plot_histogram(img_filtered, f"{FILTER_TYPE.capitalize()} Filter Response Distribution", bins=100)

---

## Step 4: Segmentation & Denoising

Aplica denoising morfológico, seguido de thresholding adaptativo para criar máscara binária do neurônio.

In [ ]:
# Grey morphological denoising
print("Applying grey morphological denoising...")
img_grey_morpho = grey_morphological_denoising(img_filtered)
# del img_filtered
# gc.collect()

# Adaptive thresholding
print("Applying adaptive mean thresholding...")
zero_t = (FILTER_TYPE != "yang")  # Yang usa threshold iterativo, outros usam > 0
img_mask, threshold_value = adaptive_mean_mask(img_grey_morpho, zero_t=zero_t)
print(f"  Threshold value: {threshold_value:.6f}")

# del img_grey_morpho
# gc.collect()

print("✓ Segmentation done!")
print_stats(img_mask.astype(float), "Binary Mask")

# Visualizar máscara
if SHOW_PROJECTIONS:
    plot_3d_projections(img_mask.astype(float), 
                       title="Neuron Mask - Max/Mean/Min Projections",
                       cmap='gray')

# Comparação: volume original com overlay da máscara
slice_idx = SLICE_TO_SHOW if SLICE_TO_SHOW else volume.shape[0] // 2
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].imshow(volume[slice_idx], cmap='gray')
axes[0].set_title('Original Volume')
axes[0].axis('off')

axes[1].imshow(img_mask[slice_idx], cmap='gray')
axes[1].set_title('Binary Mask')
axes[1].axis('off')

# Overlay
overlay = volume[slice_idx].copy()
overlay_colored = np.stack([overlay, overlay, overlay], axis=-1)
overlay_colored = overlay_colored / overlay_colored.max() if overlay_colored.max() > 0 else overlay_colored
mask_colored = np.zeros_like(overlay_colored)
mask_colored[img_mask[slice_idx], 0] = 1  # Red channel
blended = 0.7 * overlay_colored + 0.3 * mask_colored
axes[2].imshow(blended)
axes[2].set_title('Overlay (Original + Mask)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

---

## Step 5: Distance Fields (Pressure & Thrust)

Calcula campos de distância:
- **Pressure field**: distância de cada voxel até a borda (identifica centro dos neuritos)
- **Thrust field**: distância de cada voxel até o ponto raiz (identifica pontos terminais)

In [ ]:
# Criar objeto DistanceFields
print(f"Computing distance fields with root at {ROOT_COORD}...")
df = DistanceFields(
    shape=volume.shape,
    seed_point=ROOT_COORD,
    dataset_number=DATASET_NUMBER
)

# Calcular pressure field (com suavização Gaussiana)
print("Computing pressure field...")
pressure_field_raw = df.pressure_field(img_mask)
pressure_field = ndi.gaussian_filter(pressure_field_raw, 2.0)

# Calcular thrust field (com suavização Gaussiana)
print("Computing thrust field...")
thrust_field_raw = df.thrust_field(img_mask)
thrust_field = ndi.gaussian_filter(thrust_field_raw, 1.0)

# gc.collect()
print("✓ Distance fields computed!")
print_stats(pressure_field, "Pressure Field")
print_stats(thrust_field, "Thrust Field")

# Visualizar pressure field
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

slice_idx = SLICE_TO_SHOW if SLICE_TO_SHOW else volume.shape[0] // 2
im1 = axes[0].imshow(pressure_field[slice_idx], cmap='plasma')
axes[0].set_title(f'Pressure Field (Slice {slice_idx})\nDistance from boundary')
axes[0].axis('off')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(thrust_field[slice_idx], cmap='viridis')
axes[1].set_title(f'Thrust Field (Slice {slice_idx})\nDistance from root')
axes[1].axis('off')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

# Projeções 3D dos campos
if SHOW_PROJECTIONS:
    plot_3d_projections(pressure_field, "Pressure Field - Max/Mean/Min Projections", cmap='plasma')
    plot_3d_projections(thrust_field, "Thrust Field - Max/Mean/Min Projections", cmap='viridis')

---

## Step 6: Terminal Points Detection

Detecta pontos terminais do neurônio como maximas locais no thrust field.

In [ ]:
# Encontrar maxima locais no thrust field
print(f"Finding local maxima (order={MAXIMAS_MIN_DIST})...")
maximas_set = df.find_thrust_maxima(thrust_field, img_mask, order=MAXIMAS_MIN_DIST)

print(f"✓ Found {len(maximas_set)} terminal points!")
print(f"  Maxima coordinates shape: {maximas_set.shape}")
print(f"  Sample coordinates (first 5):")
for i, coord in enumerate(maximas_set[:5]):
    print(f"    {i+1}. {tuple(coord)}")

# Visualizar maxima sobre thrust field
slice_idx = SLICE_TO_SHOW if SLICE_TO_SHOW else volume.shape[0] // 2

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Thrust field
im0 = axes[0].imshow(thrust_field[slice_idx], cmap='viridis')
axes[0].set_title(f'Thrust Field (Slice {slice_idx})')
axes[0].axis('off')
plt.colorbar(im0, ax=axes[0])

# Maxima nesta fatia
maximas_in_slice = maximas_set[maximas_set[:, 0] == slice_idx]
axes[1].imshow(thrust_field[slice_idx], cmap='viridis')
axes[1].scatter(maximas_in_slice[:, 2], maximas_in_slice[:, 1], 
               c='red', s=50, marker='x', linewidths=2)
axes[1].set_title(f'Local Maxima (Slice {slice_idx})\n{len(maximas_in_slice)} points')
axes[1].axis('off')

# Overlay com volume original
axes[2].imshow(volume[slice_idx], cmap='gray')
axes[2].scatter(maximas_in_slice[:, 2], maximas_in_slice[:, 1], 
               c='red', s=50, marker='x', linewidths=2)
axes[2].plot(ROOT_COORD[2], ROOT_COORD[1], 'go', markersize=4, label='Root')
axes[2].legend()
axes[2].set_title(f'Terminal Points on Original (Slice {slice_idx})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Distribuição espacial dos maxima
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131)
ax1.hist(maximas_set[:, 0], bins=30, edgecolor='black')
ax1.set_xlabel('Z coordinate')
ax1.set_ylabel('Count')
ax1.set_title('Terminal Points Distribution (Z-axis)')
ax1.grid(alpha=0.3)

ax2 = fig.add_subplot(132)
ax2.hist(maximas_set[:, 1], bins=30, edgecolor='black')
ax2.set_xlabel('Y coordinate')
ax2.set_ylabel('Count')
ax2.set_title('Terminal Points Distribution (Y-axis)')
ax2.grid(alpha=0.3)

ax3 = fig.add_subplot(133)
ax3.hist(maximas_set[:, 2], bins=30, edgecolor='black')
ax3.set_xlabel('X coordinate')
ax3.set_ylabel('Count')
ax3.set_title('Terminal Points Distribution (X-axis)')
ax3.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## Step 7: Skeleton Generation

Usa algoritmo de Dijkstra para traçar caminhos do ponto raiz até todos os pontos terminais através do centro dos neuritos.

In [ ]:
# Gerar skeleton usando Dijkstra
print("Generating skeleton from terminal points...")
skel_coords = generate_skeleton_from_seed(
    maximas_set=maximas_set,
    seed_point=ROOT_COORD,
    pressure_field=pressure_field,
    neuron_mask=img_mask,
    shape=volume.shape,
    dataset_number=DATASET_NUMBER
)

print(f"✓ Skeleton generated: {len(skel_coords)} voxels")

# Criar imagem binária do skeleton
skel_img = create_maxima_image(skel_coords, volume.shape)
print(f"  Skeleton image non-zero voxels: {np.count_nonzero(skel_img)}")

# Aplicar skeletonize para refinar
print("Refining skeleton with morphological skeletonization...")
clean_skel = skeletonize(skel_img)
print(f"  Refined skeleton non-zero voxels: {np.count_nonzero(clean_skel)}")

# del img_mask, thrust_field, skel_img, skel_coords, maximas_set
# gc.collect()

if not np.any(clean_skel):
    print("⚠️ WARNING: Empty skeleton! Check parameters.")
else:
    print("✓ Skeleton refinement done!")

# Visualizar skeleton
slice_idx = SLICE_TO_SHOW if SLICE_TO_SHOW else volume.shape[0] // 2

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Skeleton puro
axes[0].imshow(clean_skel[slice_idx], cmap='gray')
axes[0].set_title(f'Skeleton (Slice {slice_idx})')
axes[0].axis('off')

# Overlay com pressure field
axes[1].imshow(pressure_field[slice_idx], cmap='plasma', alpha=0.7)
skel_overlay = clean_skel[slice_idx].astype(float)
axes[1].imshow(skel_overlay, cmap='Greens', alpha=0.7)
axes[1].plot(ROOT_COORD[2], ROOT_COORD[1], 'go', markersize=4, label='Root')
axes[1].legend()
axes[1].set_title(f'Skeleton + Pressure Field (Slice {slice_idx})')
axes[1].axis('off')

# Overlay com volume original
axes[2].imshow(volume[slice_idx], cmap='gray', alpha=0.8)
axes[2].imshow(skel_overlay, cmap='hot', alpha=0.6)
axes[2].plot(ROOT_COORD[2], ROOT_COORD[1], 'go', markersize=4, label='Root')
axes[2].legend()
axes[2].set_title(f'Skeleton on Original Volume (Slice {slice_idx})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Projeções 3D do skeleton
if SHOW_PROJECTIONS:
    plot_3d_projections(clean_skel.astype(float), 
                       "Skeleton - Max/Mean/Min Projections",
                       cmap='hot')

---

## Step 8: Graph Construction & MST

Cria grafo a partir do skeleton e calcula Minimum Spanning Tree (MST) para conectar todos os pontos.

In [ ]:
# Encontrar ponto raiz válido no skeleton
skel_points = np.argwhere(clean_skel)
distances_sq = np.sum((skel_points - np.array(ROOT_COORD)) ** 2, axis=1)
initial_valid_root = tuple(skel_points[np.argmin(distances_sq)])
print(f"Valid root on skeleton: {initial_valid_root}")

# Criar grafo
print("Creating graph...")
g = Graph(clean_skel, initial_valid_root)

# Calcular MST
print("Computing Minimum Spanning Tree...")
g.calculate_mst()

print(" MST computed!")
print(f"  Number of nodes: {g.mst.number_of_nodes()}")
print(f"  Number of edges: {g.mst.number_of_edges()}")

# Aplicar pruning se configurado
if PRUNING_THRESHOLD > 0:
    print(f"\nApplying pruning (threshold={PRUNING_THRESHOLD} nodes)...")
    nodes_before = g.mst.number_of_nodes()
    edges_before = g.mst.number_of_edges()
    
    g.prune_mst_by_length(PRUNING_THRESHOLD)
    
    nodes_after = g.mst.number_of_nodes()
    edges_after = g.mst.number_of_edges()
    
    print(f"  Nodes: {nodes_before} → {nodes_after} (removed {nodes_before - nodes_after})")
    print(f"  Edges: {edges_before} → {edges_after} (removed {edges_before - edges_after})")
    
    # Verificar se root ainda está no grafo
    if not g.mst.has_node(g.root):
        print("  ⚠️ Root was pruned! Finding new root...")
        import networkx as nx
        if g.mst.number_of_nodes() > 0:
            main_component = max(nx.connected_components(g.mst), key=len)
            nodes_in_component = np.array(list(main_component))
            distances_to_original = np.sum((nodes_in_component - np.array(ROOT_COORD)) ** 2, axis=1)
            new_root = tuple(nodes_in_component[np.argmin(distances_to_original)])
            g.root = new_root
            print(f"  New root: {new_root}")
        else:
            print("  ⚠️ Graph is empty after pruning!")
else:
    print("\nPruning disabled (PRUNING_THRESHOLD = 0)")

# Estatísticas do grafo
if g.mst.number_of_nodes() > 0:
    degrees = [d for n, d in g.mst.degree()]
    print(f"\nGraph statistics:")
    print(f"  Degree distribution: min={min(degrees)}, max={max(degrees)}, mean={np.mean(degrees):.2f}")
    print(f"  Terminal nodes (degree=1): {sum(1 for d in degrees if d == 1)}")
    print(f"  Branch points (degree>2): {sum(1 for d in degrees if d > 2)}")

---

## Step 9: Export to SWC

Gera arquivo SWC com smoothing e interpolação de branches.

In [ ]:
# Definir arquivo de saída
import os
output_filename = f"../results_swc/OP_{DATASET_NUMBER}_reconstruction_interactive.swc"
os.makedirs("../results_swc", exist_ok=True)

# Gerar SWC com smoothing
print(f"Generating SWC file: {output_filename}")
print(f"  Smoothing factor: {SMOOTHING_FACTOR}")
print(f"  Points per branch: {NUM_POINTS_PER_BRANCH}")

if g.mst.number_of_nodes() > 0:
    success = g.generate_smoothed_swc(
        output_filename,
        pressure_field,
        smoothing_factor=SMOOTHING_FACTOR,
        num_points_per_branch=NUM_POINTS_PER_BRANCH
    )
    
    if success:
        print(f"✓ SWC file saved successfully!")
        
        # Ler e mostrar primeiras linhas do arquivo
        with open(output_filename, 'r') as f:
            lines = f.readlines()
            print(f"\nFile preview (first 10 lines):")
            for line in lines[:10]:
                print(f"  {line.rstrip()}")
            print(f"  ... ({len(lines)} total lines)")
    else:
        print("⚠️ Failed to save SWC file")
else:
    print("⚠️ Cannot export: graph is empty")
    
# del pressure_field, g, clean_skel
# gc.collect()

---

## Pipeline Completo!

### Resumo dos Resultados

Execute a célula abaixo para ver um sumário visual de todo o pipeline:

In [ ]:
print("=" * 80)
print("CEND PIPELINE - EXECUTION SUMMARY")
print("=" * 80)
print(f"\n📁 Dataset: {DATA_DIR}")
print(f"🔧 Filter: {FILTER_TYPE}")
print(f"📐 Volume shape: {volume.shape}")
print(f"📍 Root coordinate: {ROOT_COORD}")
print(f"\n✓ Pipeline completed successfully!")
print(f"   Output file: {output_filename}")
print("=" * 80)

---

## Experimentação e Ajustes

### Dicas para Ajustar Parâmetros:

1. **Se o skeleton está muito esparso ou faltando branches:**
   - Diminua `NEURON_THRESHOLD` (ex: 0.03 ao invés de 0.05)
   - Aumente `SIGMA_MAX` para capturar neuritos mais grossos
   - Diminua `MAXIMAS_MIN_DIST` para detectar mais terminais

2. **Se há muitos false positives (ruído detectado como neurônio):**
   - Aumente `NEURON_THRESHOLD` (ex: 0.07 ou 0.1)
   - Aumente `PRUNING_THRESHOLD` para remover branches curtos

3. **Se o skeleton não conecta bem:**
   - Verifique se `ROOT_COORD` está dentro do neurônio
   - Aumente suavização dos campos (sigma no gaussian_filter)
   - Tente outro tipo de filtro (`FILTER_TYPE = "frangi"` ou `"kumar"`)

4. **Para visualizações melhores:**
   - Habilite `SHOW_PROJECTIONS = True` para ver todo o volume
   - Ajuste `SLICE_TO_SHOW` para diferentes fatias
   - Use os histogramas para entender a distribuição de intensidades

### Células para Experimentação Rápida:

Execute as células abaixo para testar funções individuais ou visualizações customizadas:

### Célula de Teste - Visualizar Diferentes Fatias

In [ ]:
# Testar visualização de diferentes fatias
test_slices = [20, 40, 60, 80]  # Ajuste conforme seu volume

fig, axes = plt.subplots(len(test_slices), 3, figsize=(15, 5*len(test_slices)))

for i, slice_idx in enumerate(test_slices):
    if slice_idx < volume.shape[0]:
        # Original
        axes[i, 0].imshow(volume[slice_idx], cmap='gray')
        axes[i, 0].set_title(f'Original - Slice {slice_idx}')
        axes[i, 0].axis('off')
        
        # Com máscara (se ainda disponível)
        # axes[i, 1].imshow(volume[slice_idx], cmap='gray')
        # axes[i, 1].imshow(img_mask[slice_idx], cmap='hot', alpha=0.3)
        axes[i, 1].set_title(f'Mask Overlay - Slice {slice_idx}')
        axes[i, 1].axis('off')
        axes[i, 1].text(0.5, 0.5, 'Run mask cell first', ha='center', va='center', 
                       transform=axes[i, 1].transAxes, fontsize=12)
        
        # Com skeleton (se ainda disponível)
        # axes[i, 2].imshow(volume[slice_idx], cmap='gray')
        # axes[i, 2].imshow(clean_skel[slice_idx], cmap='hot', alpha=0.6)
        axes[i, 2].set_title(f'Skeleton Overlay - Slice {slice_idx}')
        axes[i, 2].axis('off')
        axes[i, 2].text(0.5, 0.5, 'Run skeleton cell first', ha='center', va='center',
                       transform=axes[i, 2].transAxes, fontsize=12)

plt.tight_layout()
plt.show()

print("💡 Tip: Para ver overlays reais, re-execute as células de máscara e skeleton")

### Célula de Teste - Comparar Diferentes Filtros

Execute esta célula para comparar visualmente diferentes tipos de filtros tubulares:

In [ ]:
# Comparar diferentes filtros (ATENÇÃO: pode ser lento!)
print("⏱️ Comparando filtros... isso pode demorar um pouco")

filter_types = ["yang", "frangi", "kumar"]
results = {}

for ftype in filter_types:
    print(f"  Testing {ftype}...")
    filtered = multiscale_filtering(
        volume=volume,
        sigma_range=(SIGMA_MIN, SIGMA_MAX, SIGMA_STEP),
        filter_type=ftype,
        neuron_threshold=NEURON_THRESHOLD,
        dataset_number=DATASET_NUMBER
    )
    results[ftype] = filtered

# Visualizar comparação
slice_idx = SLICE_TO_SHOW if SLICE_TO_SHOW else volume.shape[0] // 2
fig, axes = plt.subplots(1, len(filter_types) + 1, figsize=(20, 5))

axes[0].imshow(volume[slice_idx], cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

for i, ftype in enumerate(filter_types):
    axes[i+1].imshow(results[ftype][slice_idx], cmap='hot')
    axes[i+1].set_title(f'{ftype.capitalize()} Filter')
    axes[i+1].axis('off')

plt.tight_layout()
plt.show()

# Estatísticas comparativas
print("\n📊 Filter Comparison Statistics:")
for ftype in filter_types:
    print(f"\n{ftype.upper()}:")
    print(f"  Non-zero voxels: {np.count_nonzero(results[ftype])}")
    print(f"  Mean response: {results[ftype].mean():.6f}")
    print(f"  Max response: {results[ftype].max():.6f}")
    print(f"  Std: {results[ftype].std():.6f}")

# del results
# gc.collect()
print("\n✓ Comparison done!")

---

## Recursos Adicionais

### Documentação da Nova Estrutura

Após a reestruturação do projeto, os módulos estão organizados em:

```python
from cend.core import (
    DistanceFields,           # Distance fields manager
    filters,                  # Filtros tubulares (Yang, Frangi, Kumar, Sato)
    segmentation,             # Thresholding e denoising
    skeletonization,          # Geração de skeleton com Dijkstra
)

from cend.structures import (
    Graph,                    # Grafo e MST
    SWCFile,                  # Formato SWC
)

from cend.io import (
    load_3d_volume,          # Carregar TIFF stacks
    save_3d_volume,          # Salvar TIFF stacks
)

from cend.processing import (
    multiscale_filtering,    # Filtragem multi-escala
    process_image,           # Pipeline completo
)

from cend.visualization import (
    plot_projections,        # Visualizações
)
```

### Arquivos Gerados

- **SWC file**: `../results_swc/OP_{DATASET_NUMBER}_reconstruction_interactive.swc`
- **Formato SWC**: Cada linha é `node_id type x y z radius parent_id`

### Próximos Passos

1. **Avaliar qualidade**: Compare com gold standard usando DiademMetric
2. **Ajustar parâmetros**: Use este notebook para testar diferentes configurações
3. **Batch processing**: Use o CLI (`cend`) para processar múltiplos datasets
4. **Visualização 3D**: Use os notebooks de visualização com Open3D

### Referências

- Yang et al. (2013) - DF-Tracing method
- Frangi et al. (1998) - Vesselness filter
- Kumar et al. (2013) - MVEF method

---

Happy neuron tracing!